In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline

import warnings;
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./wear_TOP100_men.csv')

In [3]:
df.head()

,Unnamed: 0,user_ID,user_Fans,outfit_ID,outfit_Url,outfit_Like,outfit_Like_adjust,outfit_Update,outfit_Season,color,outfit_Bodypart,outfit_Itemgroup,outfit_Item
0,0,genji,773507,18416506,https://wear.tw/genji/18416506/,446,446.0,2021-01-09,winter,黑色,outerwear,夾克／外套,黑色西裝大衣
1,1,genji,773507,18416506,https://wear.tw/genji/18416506/,446,446.0,2021-01-09,winter,白色,top,上衣,白色T恤
2,2,genji,773507,18416506,https://wear.tw/genji/18416506/,446,446.0,2021-01-09,winter,棕色,top,上衣,棕色針織衫
3,3,genji,773507,18416506,https://wear.tw/genji/18416506/,446,446.0,2021-01-09,winter,黑色,buttom,褲子,黑色西裝休閒褲
4,4,genji,773507,18416506,https://wear.tw/genji/18416506/,446,446.0,2021-01-09,winter,銀色,acc,首飾,銀色耳環


In [4]:
df.shape

(176154, 13)

In [5]:
#check null value
df.isnull().sum()

Unnamed: 0               0
user_ID                  0
user_Fans                0
outfit_ID                0
outfit_Url               0
outfit_Like              0
outfit_Like_adjust       0
outfit_Update            0
outfit_Season            0
color                 4888
outfit_Bodypart          0
outfit_Itemgroup         0
outfit_Item              0
dtype: int64

In [6]:
df.dropna(axis = 0, how ='any',inplace=True) 
df.isnull().sum()

Unnamed: 0            0
user_ID               0
user_Fans             0
outfit_ID             0
outfit_Url            0
outfit_Like           0
outfit_Like_adjust    0
outfit_Update         0
outfit_Season         0
color                 0
outfit_Bodypart       0
outfit_Itemgroup      0
outfit_Item           0
dtype: int64

In [7]:
df.shape

(171266, 13)

In [8]:
users = df["user_ID"].unique().tolist()
len(users)

100

In [9]:
print(users)

['genji', 'hellokeisuk', 'nosukeeee0217', 'syun0809', 'sei0', 'murasakibaru', 'bandierak26', 'kosu813ke', 'takeuma45', 'ninnosuke', 'naruyuu', 'gionryo', 'cqn15bb', 'naaaasa', 'zizz', 'matsumotokoh', 'yukihiro1126', 'sensenakajima', 'massuhair', 'wear4981', 'taisei0405', 'yoshiojazzmaster', 'yyyyuuuma', 'taisei1022', 'flocondeneige', 'makisato', 'diego1225', 'matuni', 'mrszk', 'aadgjmptwp0aaad', 'yutaka07', 'ultrabra1n', 'naoshi704', '1keino', 'crewtiger', 'kaz0608', 'fankycom104', 'oicho', 'kaz2868', '111124', 'akiiiio04', 'tapoooo', 'slvrtetsuya', 'oyasumi73i', 'byunxbyun', 'leyline0629', 'vivacematsu', 'moyammk', 'trm0303', 'yugowear11', 'shima0331', 'ryohei1026', 'tkmyuw', '1998319', 'highflyace', 'attachmentstaff', 'yuhi2486', 'hide9002', 'wfymkn2410', 'fhcguufghh', 'bbbbb0514', 'yukiskuma', 'dimda210', 'tm9tm2tm7', '90u10', '1093tokusantv', 'masaki04260', 'naokimatsubara', 'wearyuuto46', 'jiinmy', '19901204', 'yasoyaso531', '32083215', 'motoharumori1211', 'takumi00bsk', 'lsp2015'

In [10]:
df['outfit_Item']= df['outfit_Item'].astype(str)

In [11]:
#split data into train and validation dataset according to the upload dataset(history) for each model
random.shuffle(users)
users_train = [users[i] for i in range(round(0.8*len(users)))]
train_df = df[df['user_ID'].isin(users_train)]
validation_df = df[~df['user_ID'].isin(users_train)]

In [12]:
# list to capture upload history of the users
upload_train = []
for i in tqdm(users_train):
    temp = train_df[train_df["user_ID"] == i]["outfit_Item"].tolist()
    upload_train.append(temp)

print(upload_train[0])

100%|██████████| 100/100 [00:02<00:00, 42.92it/s]

['白色針織衫', '灰色褲裝', '黑色靴子', '黑色夾克／外套', '黑色褲子', '黑色球鞋', '黑色褲子', '黑色靴子', '黑色褲子', '黑色鞋子', '黑色夾克／外套', '白色襯衫', '黑色褲子', '黑色鞋子', '黑色夾克／外套', '黑色針織衫', '黑色針織衫', '黑色褲子', '黑色鞋子', '黑色毛綫帽', '黑色夾克／外套', '棕色連帽外套', '黑色夾克／外套', '黑色褲子', '黑色球鞋', '黑色帽子', '黑色針織衫', '黑色外套', '黑色褲子', '黑色靴子', '灰色褲裝', '灰色外套', '白色針織衫', '黑色靴子', '棕色針織衫', '黑色夾克／外套', '米色褲裝', '黑色靴子', '白色針織衫', '黑色夾克／外套', '黑色褲子', '黑色靴子', '黑色褲子', '棕色針織衫', '藍色短外套', '黑色靴子', '白色針織衫', '黑色夾克／外套', '黑色褲子', '黑色靴子', '棕色針織衫', '白色襯衫', '藍色牛仔褲', '黑色球鞋', '黑色夾克／外套', '灰色外套', '灰色褲裝', '黑色球鞋', '黑色帽子', '黑色褲子', '黑色靴子', '黑色貝雷帽', '紅色針織衫', '黑色夾克／外套', '黑色褲子', '黑色靴子', '黑色夾克／外套', '棕色連帽外套', '米色褲裝', '黑色球鞋', '棕色針織衫', '黑色褲子', '黑色靴子', '黑色夾克／外套', '黑色針織衫', '黑色褲子', '黑色鞋子', '灰色針織衫', '黑色外套', '黑色褲子', '黑色靴子', '黑色褲子', '黑色球鞋', '黑色針織衫', '黑色夾克／外套', '白色針織衫', '藍色短外套', '黑色褲裝', '黑色球鞋', '紅色針織衫', '黑色褲子', '黑色靴子', '灰色針織衫', '灰色外套', '灰色褲裝', '黑色靴子', '黑色夾克／外套', '黑色褲子', '黑色鞋子', '白色針織衫', '棕色連帽外套', '黑色褲子', '黑色球鞋', '黑色夾克／外套', '黑色褲子', '黑色靴子', '黑色夾克／外套', '灰色針織衫', '藍色牛仔褲', '黑色靴子', '黑色褲子', '黑色針織衫', '黑色外套', '黑色靴子', '黑色夾克／

In [13]:
upload_val = []
for i in tqdm(validation_df['user_ID'].unique()):
    temp = validation_df[validation_df["user_ID"] == i]["outfit_Item"].tolist()
    upload_val.append(temp)

In [13]:
# train word2vec model
model = Word2Vec(window = 2, sg = 1, hs = 0, #window=2
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(upload_train, progress_per=200)

model.train(upload_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

(983817, 1712660)

In [14]:
model.save("word2vec_wear.model")

In [15]:
print(model)

Word2Vec(vocab=699, size=100, alpha=0.03)


In [16]:
# extract all vectors
X = model[model.wv.vocab]
X.shape

(699, 100)

In [17]:
#remove duplicates
outfit_Url=train_df['outfit_Url']
outfit_Url.drop_duplicates(inplace=True, keep="last")
outfit_Url.shape

(39989,)

In [18]:
# create product-ID and product-description dictionary
outfit_info = train_df[["outfit_Item", "outfit_Url","outfit_Like_adjust","outfit_Season","outfit_Bodypart"]]
outfit_info.sort_values(by=['outfit_Like_adjust'], inplace=True,ascending=False)
outfit_dict = outfit_info.groupby(['outfit_Season','outfit_Item'])['outfit_Url'].apply(list).to_dict()

In [19]:
# Make an Recommendatin for most similar goods 
def similar_products(season,v, n = 4):
    m =  model.similar_by_vector(v, topn= 4)[0]
    m_recommend_outfitUrl=(m[0],outfit_dict[season,m[0]][:3] , m[1])
    
    # extract most similar products for the input vector
    ms = model.similar_by_vector(v, topn= 10)[1:]
    ms_recommend_outfitUrl = []
    for j in ms:   
        try:
            pair = (j[0],outfit_dict[season,j[0]][:3] , j[1])
            ms_recommend_outfitUrl.append(pair)
        except:
            continue
    return m_recommend_outfitUrl, ms_recommend_outfitUrl

In [20]:
similar_products('winter',model['米色休閒長褲'])

(('米色休閒長褲',
  ['https://wear.tw/vivacematsu/18178920/',
   'https://wear.tw/vivacematsu/18314541/',
   'https://wear.tw/vivacematsu/18350051/'],
  1.0),
 [('棕色休閒長褲',
   ['https://wear.tw/byunxbyun/16020845/',
    'https://wear.tw/byunxbyun/18200612/',
    'https://wear.tw/howellmhl/18447742/'],
   0.7037085294723511),
  ('藍色牛仔褲',
   ['https://wear.tw/taisei0405/16027046/',
    'https://wear.tw/1228masa/18324379/',
    'https://wear.tw/hellokeisuk/16288629/'],
   0.6776266098022461),
  ('銀色手提包', ['https://wear.tw/highflyace/11667505/'], 0.604658842086792),
  ('灰色休閒長褲',
   ['https://wear.tw/mrszk/13984673/',
    'https://wear.tw/cqn15bb/15985503/',
    'https://wear.tw/lsp2015/18328355/'],
   0.6037817001342773),
  ('綠色休閒短褲',
   ['https://wear.tw/naruyuu/16076396/',
    'https://wear.tw/ninnosuke/18315567/',
    'https://wear.tw/repopoo/16112456/'],
   0.5885773301124573),
  ('藍色西裝褲',
   ['https://wear.tw/97_tanaka/18201473/',
    'https://wear.tw/ultrabra1n/15975791/'],
   0.58038198947

In [21]:
# Make an Recommendatin for fashion ensemble
def similar_products_byitem(season,v, n = 4):
    m =  model.similar_by_vector(v, topn= 4)[0]
    m_recommend_outfitUrl=(m[0],outfit_dict[season,m[0]][:3] , m[1])
    
    # extract most similar products for the input vector
    ms = model.similar_by_vector(v, topn= 100)[1:]
    ms_recommend_outfitUrl = []
    for j in ms:
        if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==m[0]].values[0]=='top':
            if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==j[0]].values[0] \
            in ['outerear','buttom']:
                try:
                    pair = (j[0],outfit_dict['summer',j[0]][:3] , j[1])
                    ms_recommend_outfitUrl.append(pair)
                except:
                    continue
        elif outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==m[0]].values[0]=='buttom':
            if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==j[0]].values[0] \
            in ['top','outerwear']:
                try:
                    pair = (j[0],outfit_dict['summer',j[0]][:3] , j[1])
                    ms_recommend_outfitUrl.append(pair)
                except:
                    continue
        elif outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==m[0]].values[0]=='outerwear':
            if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==j[0]].values[0] \
            in ['top','buttom','dress']:
                try:
                    pair = (j[0],outfit_dict['summer',j[0]][:3] , j[1])
                    ms_recommend_outfitUrl.append(pair)
                except:
                    continue
        elif outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==m[0]].values[0]=='dress':
            if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==j[0]].values[0] \
            in ['outerwear']:
                try:
                    pair = (j[0],outfit_dict['summer',j[0]][:3] , j[1])
                    ms_recommend_outfitUrl.append(pair)   
                except:
                    continue
    return m_recommend_outfitUrl, ms_recommend_outfitUrl

In [22]:
similar_products_byitem('summer',model['白色褲裝'])

(('白色褲裝',
  ['https://wear.tw/1228masa/17235257/',
   'https://wear.tw/takeuma45/17031568/',
   'https://wear.tw/hellokeisuk/16965707/'],
  1.0),
 [('白色休閒西裝',
   ['https://wear.tw/genji/17514622/',
    'https://wear.tw/motoharumori1211/17431356/',
    'https://wear.tw/90u10/4501597/'],
   0.5832391977310181),
  ('白色連體套裝',
   ['https://wear.tw/byunxbyun/17159036/',
    'https://wear.tw/byunxbyun/17536582/',
    'https://wear.tw/byunxbyun/17240481/'],
   0.5149715542793274)])

In [23]:
#Make an Recommendatin for fashion ensemble(Accessory)
def similar_products_byitem_acc(season,v, n = 4):
    m =  model.similar_by_vector(v, topn= 4)[0]
    m_recommend_outfitUrl=(m[0],outfit_dict[season,m[0]][:3] , m[1])
    
    # extract most similar products for the input vector, topn=向量最相似的前幾名
    ms = model.similar_by_vector(v, topn= 50)[1:]
    ms_recommend_outfitUrl = []
    for j in ms:
        if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==m[0]].values[0]=='top':
            if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==j[0]].values[0] \
            in ['outerear','buttom','acc']:
                try:
                    pair = (j[0],outfit_dict['summer',j[0]][:3] , j[1])
                    ms_recommend_outfitUrl.append(pair)
                except:
                    continue
        elif outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==m[0]].values[0]=='buttom':
            if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==j[0]].values[0] \
            in ['top','outerwear','acc']:
                try:
                    pair = (j[0],outfit_dict['summer',j[0]][:3] , j[1])
                    ms_recommend_outfitUrl.append(pair)
                except:
                    continue
        elif outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==m[0]].values[0]=='outerwear':
            if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==j[0]].values[0] \
            in ['top','buttom','dress','acc']:
                try:
                    pair = (j[0],outfit_dict['summer',j[0]][:3] , j[1])
                    ms_recommend_outfitUrl.append(pair)
                except:
                    continue
        elif outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==m[0]].values[0]=='dress':
            if outfit_info['outfit_Bodypart'][outfit_info['outfit_Item']==j[0]].values[0] \
            in ['outerwear','acc']:
                try:
                    pair = (j[0],outfit_dict['summer',j[0]][:3] , j[1])
                    ms_recommend_outfitUrl.append(pair)   
                except:
                    continue
    return m_recommend_outfitUrl, ms_recommend_outfitUrl

In [24]:
similar_products_byitem_acc('summer',model['白色褲裝'])

(('白色褲裝',
  ['https://wear.tw/1228masa/17235257/',
   'https://wear.tw/takeuma45/17031568/',
   'https://wear.tw/hellokeisuk/16965707/'],
  1.0),
 [('棕色首飾',
   ['https://wear.tw/yugowear11/17378649/',
    'https://wear.tw/yugowear11/17374426/',
    'https://wear.tw/yugowear11/17352600/'],
   0.6247581839561462),
  ('銀色胸針/胸花', ['https://wear.tw/zizz/17333743/'], 0.6238014698028564),
  ('藍色環保袋',
   ['https://wear.tw/yugowear11/17182510/',
    'https://wear.tw/yugowear11/17230431/',
    'https://wear.tw/yugowear11/17276199/'],
   0.6223949790000916),
  ('棕色時尚配件',
   ['https://wear.tw/sensenakajima/17267864/',
    'https://wear.tw/sensenakajima/17382171/',
    'https://wear.tw/sensenakajima/17261631/'],
   0.6052163243293762),
  ('棕色寬邊帽',
   ['https://wear.tw/zizz/17119200/', 'https://wear.tw/zizz/16989197/'],
   0.6013150215148926),
  ('黃色方巾',
   ['https://wear.tw/repopoo/15044152/', 'https://wear.tw/repopoo/15079594/'],
   0.5974345207214355),
  ('藍色時尚配件',
   ['https://wear.tw/matuni/150